In [1]:
from haystack.nodes import WebRetriever
from haystack.schema import Document
import os
from dotenv import load_dotenv
from typing import List 
from haystack.document_stores import PineconeDocumentStore
from haystack.document_stores import FAISSDocumentStore

# Load environment variables (if any)
load_dotenv("../.env")
serp = os.getenv("SERP_API_KEY")
pinecone_key = os.getenv("PINECONE_API")

In [3]:
# Initialize WebRetriever
# You need to replace 'your_api_key' with an actual API key
retriever = WebRetriever(api_key=serp, 
                         mode="preprocessed_documents",
                         top_k=100)

# Retrieve documents based on a query
# You might need to use a query that would result in the IMDb page you are interested in
# This is a hypothetical example, replace 'movie reviews' with a relevant query
documents : List[Document] = retriever.retrieve(query="IMDB movie reviews for the Barbie movie (2023)")



In [ ]:
document_store = PineconeDocumentStore(api_key=pinecone_key,
                                       similarity="cosine",
                                       embedding_dim=768)

In [4]:
document_store = FAISSDocumentStore(faiss_index_factory_str="Flat", return_embedding=True)


In [5]:
from haystack.nodes import  DensePassageRetriever

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)


/Users/macpro/anaconda3/envs/llm-pipelines/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
# Delete existing documents in documents store
document_store.delete_documents()

# Write documents to document store
document_store.write_documents(documents)

# Add documents embeddings to index
document_store.update_embeddings(retriever=retriever)

Writing Documents: 10000it [00:00, 11382.22it/s]         
Documents Processed: 10000 docs [00:51, 194.70 docs/s]        
